# Demo 3: Secondary Aims Using Data Arising from a SMART

**Authors:** Nicholas J. Seewald, Daniel Almirall, Inbal Nahum-Shani, Audrey Boruvka, and Susan A. Murphy

**Date:** March 20, 2020

This file provides example R code to analyze *simulated* data that was generated to mimic data arising from the ADHD SMART study (PI: William Pelham). An accompanying handout ("ADHD Handout.pdf") describes the variables in the data.

Translation of SAS code from Modules 3 and 4 into R is by Audrey Boruvka and Nicholas J. Seewald [(email)](mailto:nseewald@umich.edu). Notebooks created by [Nicholas J. Seewald](https://nickseewald.com).

### Contents:
- [Part (a): Compare means from two embedded adaptive interventions](#part-a)
- [Part (b): Compare all four embedded AIs with one regression](#part-b)

## Setup
As in [Module 3](module3tutorial.ipynb) and [Module 4](module4tutorial.ipynb), we will begin by loading the data. We omit summaries here for brevity; see [module3tutorial.ipynb](module3tutorial.ipynb) for details on the data we will be using in this tutorial. The block of code below performs all necessary pre-processing steps from Module 3 related to loading and sorting the data, as well as centering covariates. **See Module 3 for details on this code; otherwise just run the cells and move to Part (a).**

In [ ]:
library(geepack)
source('functions.R')

In [ ]:
adhd <- read.csv("adhd-simulated-dataset.csv")

# R is case-sensitive! Avoid issues by changing variable names to all lowercase
names(adhd) <- tolower(names(adhd))

# Handle missing data: set "NaN" values to "NA" (R's missing data value)
adhd$o21[is.nan(adhd$o21)] <- NA
adhd$a2[is.nan(adhd$a2)] <- NA

# Sort data by ID
adhd <- adhd[order(adhd$id), ]

adhd$o11c <- with(adhd, o11 - mean(o11))
adhd$o12c <- with(adhd, o12 - mean(o12))
adhd$o13c <- with(adhd, o13 - mean(o13))
adhd$o14c <- with(adhd, o14 - mean(o14))
## o21 is measured among non-responders only
adhd$o22c <- with(adhd, o22 - mean(o22))

## center baseline variables using mean among non-responders
adhd$o11cnr <- adhd$o12cnr <- adhd$o13cnr <- adhd$o14cnr <- NA
adhd$o21cnr <- adhd$o22cnr <- NA
adhd$o11cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o11 - mean(o11))
adhd$o12cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o12 - mean(o12))
adhd$o13cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o13 - mean(o13))
adhd$o14cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o14 - mean(o14))
adhd$o21cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o21 - mean(o21))
adhd$o22cnr[adhd$r == 0] <- with(subset(adhd, r == 0), o22 - mean(o22))

## <a name="part-a"></a> Part (a): Examine moderators of second-stage treatment effect

Remember that Step 1 of Q-learning is to understand how intermediate outcomes can be used to make second-stage decisions about intensifying vs. augmenting. Investigating potential moderators in this way will help us tailor the second-stage intervention for non-responders based on the status of the participants up to the point of non-response. We are only able to investigate non-responders since those were the only individuals who were re-randomized. Because all responders in this study continued on their first-stage intervention, we are unable to assess whether a different tactic would be better for some responders.

We'll start by fitting a **moderated regression model** using the data from non-responders. The goal is to discover if we can use *the initial treatment* `a1` and *adherence to initial treatment* `o22` to select a tactic for non-responders, adjusting for baseline covariates. The model is as follows:

$$ E[Y \mid \mathbf{O_{1}}, A_1, O_{22}, A_2, R = 0] = \beta_0 + \beta_1 O_{11c} + \beta_2 O_{12c} + \beta_3 O_{13c} + \beta_4 O_{14c} + \beta_5 O_{21c} + \beta_6 A_{1} + \beta_7 O_{22} + \beta_8 A_2 + \beta_9 (A_2 \times A_1) + \beta_{10} (A_2 \times O_{22})$$

Pay particular attention to $\beta_6$, $\beta_7$, $\beta_9$, and $\beta_{10}$, as these represent the main effects and interactions (with second-stage intervention), respectively, of the two moderators we are interested in for this analysis. Let's fit the model.

In [ ]:
# The argument "x = TRUE" requests that geeglm() return the matrix of data used to fit the model.
fit <- geeglm(y ~ o11c + o12c + o13c + o14c + o21cnr + a1 + o22 + a2
         + a1:a2 + a2:o22,
         id = id, data = adhd, subset = r == 0, x = TRUE)

Let's look at the parameter estimates and standard errors for this model to investigate moderation of the second-stage intervention effect by first-stage intervention and adherence.

Unlike `PROC GENMOD` in SAS when we have no weights, when we summarize the results from `geeglm()` it will return the "robust" standard errors (which we don't want here, since this is an unweighted regression). So, we'll use the `estimate()` function from [`functions.R`](https://notebooks.azure.com/nseewald/projects/getting-smart-workshop/html/functions.R) with an identity matrix for the `combos` argument. The identity matrix has 1's on the diagonal and 0's elsewhere.

In [ ]:
mat <- diag(ncol(fit$x)) #mat is the appropriately-sized identity matrix 
rownames(mat) <- names(coef(fit)) #name the rows of mat with variable names for legibility
estimate(fit, combos = mat)

We can see that there does appear to be statisically-significant moderation of the effect of the second-stage intervention by first-stage intervention (`a1`) and adherence (`o22`). 

Now, we proceed to estimate contrasts of interest. In particular, we're interested in estimating the mean outcome for non-responders to either first-stage intervention at both levels of adherence and first-stage intervention.

In [ ]:
estimate(fit,
  rbind("NR adh to med, intensify"          = c(1, rep(0,5), -1, 1,  1, -1,  1),
        "NR adh to med, augment"            = c(1, rep(0,5), -1, 1, -1,  1, -1),
        "Intens vs aug, NR adh to med"      = c(0, rep(0,5),  0, 0,  2, -2,  2),
        "NR non-adh to med, intensify"      = c(1, rep(0,5), -1, 0,  1, -1,  0),
        "NR non-adh to med, augment"        = c(1, rep(0,5), -1, 0, -1,  1,  0),
        "Intens vs aug, NR non-adh to med"  = c(0, rep(0,5),  0, 0,  2, -2,  0),
        "NR adh to bmod, intensify"         = c(1, rep(0,5),  1, 1,  1,  1,  1),
        "NR adh to bmod, augment"           = c(1, rep(0,5),  1, 1, -1, -1, -1),
        "Intens vs aug, NR adh to bmod"     = c(0, rep(0,5),  0, 0,  2,  2,  2),
        "NR non-adh to bmod, intensify"     = c(1, rep(0,5),  1, 0,  1,  1,  0),
        "NR non-adh to bmod, augment"       = c(1, rep(0,5),  1, 0, -1, -1,  0),
        "Intens vs aug, NR non-adh to bmod" = c(0, rep(0,5),  0, 0,  2,  2,  0)))

We can plot the contrasts below, as in the slides:

!["Plot of estimated outcomes for non-responders under both second-stage interventions, by adherence and first-stage intervention"](assets/qlearning-step1-plot.png "Plot of estimated outcomes for non-responders under both second-stage interventions, by adherence and first-stage intervention")

## <a name="part-b"></a> Part (b): Q-Learning using `qlaci`

The easiest way to perform Q-learning in R is to use a package called `qlaci`. The following line of code will install the package from [d3Lab's GitHub repository](https://github.com/d3lab-isr/qlaci), via a package called `remotes`, which has been automatically installed in this notebook environment. We'll use the `install_github()` function from remotes and provide it a GitHub repository. Here, the code repository we're interested in is `d3lab-isr/qlaci`.

In [ ]:
remotes::install_github("d3lab-isr/qlaci")

To get started with `qlaci()`, we'll need to create a `data.frame` that contains only non-responders, as these are the only individuals we can use Q-learning on. We'll call the datasest `adhdq`. 

In [ ]:
adhdq <- adhd
adhdq[is.na(adhdq)] <- 0

We also need to create a contrast matrix, as we've done for the `estimate()` steps in all prior modules. This is the contrast matrix that will be used for the *stage-1 regression* (step 3 of Q-learning). `qlaci()` uses this matrix to estimate the mean outcomes under each of the first-stage treatments (averaging over the future optimal response) at both levels of "prior med", the baseline covariate we'd like to investigate as part of a more deeply-tailored AI.

In [ ]:
## contrast matrix - we must transpose this for qlaci
c1 <-
  rbind("Mean Y under bmod, prior med"          = c(1, rep(0, 3), 1,  1,  1),
        "Mean Y under med, prior med"           = c(1, rep(0, 3), 1, -1, -1),
        "Mean diff (bmod-med) for prior med"    = c(0, rep(0, 3), 0,  2,  2),
        "Mean Y under bmod, no prior med"       = c(1, rep(0, 3), 0,  0,  1),
        "Mean Y under med, no prior med"        = c(1, rep(0, 3), 0,  0, -1),
        "Mean diff (bmod-med) for no prior med" = c(0, rep(0, 3), 0,  0,  2))

Now we're ready to jump into `qlaci()`. The first thing we need to do is "set the random seed". Random number generation in software is actually "pseudo-random" and dependent on a "seed". If you know the seed, you can determine the entire sequence of numbers that will be generated by the software. We'll set the seed at 0 so that we can reproduce the results of `qlaci()`. We need to do this since the confidence intervals from `qlaci()` are based on the bootstrap, which involves randomness.

In [ ]:
## setting the random seed ensures that the results are reproducible
set.seed(0)

The following are the arguments we need to provide to `qlaci()`:

- `H10`: Baseline covariates we want to adjust for in the first-stage regression. This should be a matrix, with each column representing data for a main-effects term in the first-stage regression. To include an intercept, add a column of 1's.
- `H11`: Variables that interact with first-stage treatment in the first-stage regression (candidate variables for deeper tailoring).As with `H10`, this should be a matrix, with each column representing a multiple of first-stage treatment `A1`. Note that this means that to include a main effect of first-stage treatment, we need to include a column of 1's.
- `A1`: Indicator for first-stage treatment
- `Y1`: A continuous intermediate outcome. Here, we don't have an intermediate outcome, so we set this to zero for everyone.
- `H20`: A matrix, with each column containing data for a main-effects term in the second-stage regression (analogous to `H10`).
- `H21`: Variables that interact with second-stage treatment `A2` in the second-stage regression. A matrix, with each column containing a multiple of `A2`. To include a main effect of second-stage treatment, include a column of 1's.
- `A2`: Indicator for second-stage treatment
- `Y2`: End-of-study outcome
- `S`: Indicator for whether an individual was re-randomized (1 = re-randomized; 0 = otherwise)
- `c1`: Contrast matrix for first-stage regression (see above)

In [ ]:
## with attach we can be lazy and refer to variables directly; use with caution
attach(adhdq)
ql <- qlaci(H10 = cbind("intercept" = 1, o11c, o12c, o14c, o13), 
            H11 = cbind("o13:a1" = o13, "a1" = 1),
            A1 = a1, 
            Y1 = rep(0, nrow(adhdq)),
            H20 = cbind("intercept" = 1, o11cnr, o12cnr, o13cnr, o14cnr, o21cnr, a1, o22),
            H21 = cbind("o22:a2" = o22, "a1:a2" = a1, "a2" = 1),
            A2 = a2, 
            Y2 = y, 
            S = rerand,
            c1 = t(c1))
detach(adhdq)
ql